In [3]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import os

In [2]:
path = os.getcwd() + '/data_sets/bank-additional-full.csv'

data = pd.read_csv(path, sep = ';')


In [3]:
var_names = data.columns.tolist()

In [4]:
#these are the categorical variables
categs = ['job','marital','education','default','housing','loan','contact','month','day_of_week','duration','poutcome','y']


#now we will extract quantitive vars

quantit = [i for i in var_names if i not in categs]

# Convert categorical variable into dummy/indicator variables

job = pd.get_dummies(data['job'])
marital = pd.get_dummies(data['marital'])
education = pd.get_dummies(data['education'])
default = pd.get_dummies(data['default'])
housing = pd.get_dummies(data['housing'])
loan = pd.get_dummies(data['loan'])
contact = pd.get_dummies(data['contact'])
month = pd.get_dummies(data['month'])
day_of_week = pd.get_dummies(data['day_of_week'])
duration = pd.get_dummies(data['duration'])
poutcome = pd.get_dummies(data['poutcome'])

# now y column still has value yes and no
#so convert it t 1 and 0

dict_map = dict()
y_map = {'yes': 1,'no':0}
dict_map['y'] = y_map

#now replace the y column in data with the new y 

data = data.replace(dict_map)
label = data['y']


In [5]:
numerical_data = data[quantit]

In [6]:
numerical_data_columns = numerical_data.columns.tolist()


In [7]:
# scale the data using minmax scaler 
scaler = preprocessing.MinMaxScaler()

scaled_data = scaler.fit_transform(numerical_data)

#convert scaled data to data frame object

scaled_data_data_frame = pd.DataFrame(scaled_data)
#assign name to colums of new data frmae object created

In [8]:
scaled_data_data_frame.columns = numerical_data_columns

In [9]:
#now concatenate all data aka categorical datat and numerical data

normalized_data = pd.concat([scaled_data_data_frame,job,marital,education,default,housing,loan,contact,month,
                            day_of_week,duration,poutcome,label],axis = 1)


In [10]:
# now save the data to csv file 

normalized_data.to_csv('bank-normalized.csv',index = False)

Now we will implement the nueral network for the above dataset

In [4]:
import tensorflow as tf
from sklearn.cross_validation import train_test_split 

/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
# First of all define all hyper-parameters

LEARNING_RATE = 0.001
NUMBER_EPOCH = 100
BATCH_SIZE = 100
DISPLAY_EPOCH = 20
N_HIDDEN_NEURON = 250
STDDEV = 0.1
ACTIVATION_FUNCTION_OUT = tf.nn.tanh
RANDOM_STATE = 100

In [6]:
# now ready the data_set we have prepared earlier

In [8]:
doc_location = os.getcwd() + '/bank_normalized.csv'
read_data = pd.read_csv(doc_location)
print("RAW DATA LOADED SUFFESSFULLY")


RAW DATA LOADED SUFFESSFULLY


now we will define data specific hyperparameters

In [9]:
Y_LABEL = 'y'

DATA_COLUMNS = [i for i in read_data.columns.tolist() if i != Y_LABEL]

M = read_data.shape[0]  # USUAL NOTATION FOR TOTAL NUMBER OF TRAINING EXAMPLES

N = read_data.shape[1] - 1 # total number of features -1 to subtract y label 

N_CLASSES = read_data[Y_LABEL].unique().shape[0]

TEST_SIZE = 0.25
TRAIN_SIZE = int(M*(1 - TEST_SIZE))


In [10]:
#NOW WE LOAD X DATA AND Y DATAT
x_data = read_data[DATA_COLUMNS].get_values()
y_data = read_data[Y_LABEL].get_values()

#one hot encoding for y_data
labels_ = np.zeros((M, N_CLASSES))
labels_[np.arange(M), y_data] = 1

x_train,x_test,y_train,y_test = train_test_split(x_data,labels_,
                                                 test_size = TEST_SIZE,
                                                 random_state = RANDOM_STATE)


In [11]:
n_input = N
n_hidden_1 = N_HIDDEN_NEURON
n_hidden_2 = N_HIDDEN_NEURON
n_hidden_3 = N_HIDDEN_NEURON
n_hidden_4 = N_HIDDEN_NEURON
n_classes = N_CLASSES
dropout_keep_prob = tf.placeholder(tf.float32)

In [12]:
X = tf.placeholder(tf.float32, shape = [None,n_input])
y = tf.placeholder(tf.float32, shape = [None, n_classes])

def mpl(_X,_weights,_biases,dropout_keep_prob):
    layer1 = tf.nn.dropout(tf.nn.tanh(tf.add(tf.matmul(_X,_weights['w1']), _biases['b1'])),dropout_keep_prob)
    layer2 = tf.nn.dropout(tf.nn.tanh(tf.add(tf.matmul(layer1,_weights['w2']), _biases['b2'])),dropout_keep_prob)
    layer3 = tf.nn.dropout(tf.nn.tanh(tf.add(tf.matmul(layer2,_weights['w3']), _biases['b3'])),dropout_keep_prob)
    layer4 = tf.nn.dropout(tf.nn.tanh(tf.add(tf.matmul(layer3,_weights['w4']), _biases['b4'])),dropout_keep_prob)
    out = ACTIVATION_FUNCTION_OUT((tf.add(tf.matmul(layer4,_weights['w5']), _biases['b5'])))
    return out

weights = {
    'w1' : tf.Variable(tf.random_normal([n_input,n_hidden_1],stddev = STDDEV)),
    'w2' : tf.Variable(tf.random_normal([n_hidden_1,n_hidden_2],stddev = STDDEV)),
    'w3' : tf.Variable(tf.random_normal([n_hidden_2,n_hidden_3],stddev = STDDEV)),
    'w4' : tf.Variable(tf.random_normal([n_hidden_3,n_hidden_4],stddev = STDDEV)),
    'w5' : tf.Variable(tf.random_normal([n_hidden_4,n_classes],stddev = STDDEV))
}

biases = {
    'b1' : tf.Variable(tf.random_normal([n_hidden_1],stddev = STDDEV)),
    'b2' : tf.Variable(tf.random_normal([n_hidden_2],stddev = STDDEV)),
    'b3' : tf.Variable(tf.random_normal([n_hidden_3],stddev = STDDEV)),
    'b4' : tf.Variable(tf.random_normal([n_hidden_4],stddev = STDDEV)),
    'b5' : tf.Variable(tf.random_normal([n_classes],stddev = STDDEV))
    
}



In [13]:
#here the mathematical calculateions beware
predict = mpl(X,weights,biases,dropout_keep_prob)

cost_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = predict, labels = y))

optimizer = tf.train.AdamOptimizer(learning_rate = LEARNING_RATE).minimize(cost_op)


In [14]:
# now compute the accuracy of 

correct_prediction = tf.equal(tf.argmax(predict,1),tf.argmax(y,1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
print("DEEP NEURAL NETWORD HAS BEEN BUILT SUCCESSFULLY")
print("STARTED TRAINING")


DEEP NEURAL NETWORD HAS BEEN BUILT SUCCESSFULLY
STARTED TRAINING


In [16]:
#NOW THE TYPICAL TENSORFLOWS JOB

init_op = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init_op)
    # this one is the traningloop
    for i in range(NUMBER_EPOCH):
        avg_cost = 0
        total_batch = int(x_train.shape[0]//BATCH_SIZE)
        
        # this one is to loop over all batches
        for j in range(total_batch):
            rand_id_x = np.random.randint(int(TRAIN_SIZE), size = BATCH_SIZE)
            batch_xs = x_train[rand_id_x, :]
            batch_ys = y_train[rand_id_x,:]
            
            
            
            #now fit this randomly indexes batched data
            sess.run(optimizer, feed_dict={X:batch_xs,y:batch_ys,dropout_keep_prob:0.9})
            
            avg_cost+= sess.run(cost_op,feed_dict={X:batch_xs,y:batch_ys,dropout_keep_prob:0.9})/total_batch
            
        if i % DISPLAY_EPOCH ==0:
            print("Epoch %03d/%03d cost : %.9f" % (i,NUMBER_EPOCH,avg_cost))
            train_acc = sess.run(accuracy,feed_dict={X:batch_xs, y:batch_ys, dropout_keep_prob:1.})
            print("training accuracy %.3f" % (train_acc))
    print("End of training.\n")
    print("Testing....\n")
    
    #now test the data
    
    test_acc = sess.run(accuracy,feed_dict={X:x_test, y: y_test,dropout_keep_prob:1.})
    print("Test accuracy: %.3f" % (test_acc))
    print("Your mlp model has beel successfuly trained")
    
        
        

Epoch 000/100 cost : 0.214328094
training accuracy 0.980
Epoch 020/100 cost : 0.209041875
training accuracy 0.970
Epoch 040/100 cost : 0.208228864
training accuracy 0.960
Epoch 060/100 cost : 0.208879270
training accuracy 0.970
Epoch 080/100 cost : 0.216196346
training accuracy 0.970
End of training.

Testing....

Test accuracy: 0.956
Your mlp model has beel successfuly trained


11